In [1]:
from syft_rds.orchestra import setup_rds_stack

In [2]:
stack = setup_rds_stack(log_level="INFO")

2025-03-05 11:44:45.335 | WARNING  | syft_rds.orchestra:setup_rds_stack:89 - Using shared data directory for both clients. Any file permission checks will be skipped as both clients have access to the same files.
2025-03-05 11:44:45.339 | INFO     | syft_rds.orchestra:setup_rds_stack:112 - Launching mock RDS stack in /var/folders/6x/3xzhslqj2tx0sgx88x022qyw0000gp/T/rds_o1308ll1...


In [3]:
do_client = stack.do_rds_client
ds_client = stack.ds_rds_client

# Create a dataset as DO

In [4]:
import os
# make a mock dataset

os.makedirs("dataset-1/private", exist_ok=True)
os.makedirs("dataset-1/mock", exist_ok=True)
with open("dataset-1/private/data.csv", "w") as f:
    f.write("-1,-2,-3")

with open("dataset-1/mock/data.csv", "w") as f:
    f.write("1,2,3")


In [5]:
data = do_client.dataset.create(
    name="dataset-1",  # MUST BE UNIQUE. Throw Exception if already exist.
    path="/Users/azizwork/Workspace/rds/notebooks/dataset-1/private/",  # MUST EXIST
    mock_path="/Users/azizwork/Workspace/rds/notebooks/dataset-1/mock/",
)

In [6]:
do_client.datasets

[Dataset
   uid: 482fbc77-45a7-4aeb-91f3-e83c9636e9d9
   created_at: 2025-03-05T10:44:50.030997Z
   updated_at: 2025-03-05T10:44:50.031001Z
   name: dataset-1
   private: syft://data_owner@test.openmined.org/private/datasets/dataset-1
   mock: syft://data_owner@test.openmined.org/public/datasets/dataset-1
   summary: None
   readme: None
   tags: []]

# Use dataset as DS

In [7]:
ds_client.datasets

[Dataset
   uid: 482fbc77-45a7-4aeb-91f3-e83c9636e9d9
   created_at: 2025-03-05T10:44:50.030997Z
   updated_at: 2025-03-05T10:44:50.031001Z
   name: dataset-1
   private: syft://data_owner@test.openmined.org/private/datasets/dataset-1
   mock: syft://data_owner@test.openmined.org/public/datasets/dataset-1
   summary: None
   readme: None
   tags: []]

In [8]:
ds_client.dataset.get("dataset-1").get_mock_path()

PosixPath('/private/var/folders/6x/3xzhslqj2tx0sgx88x022qyw0000gp/T/rds_o1308ll1/shared_client_dir/datasites/data_owner@test.openmined.org/public/datasets/dataset-1')

In [10]:
dataset = ds_client.dataset.get("dataset-1")
dataset.describe()

└── data.csv


True

In [17]:
import pathlib
from syft_rds.models.models import Dataset
from syft_rds.utils.render_code_template import render_data_access_template


def init_code(dataset: Dataset, path: str) -> None:
    path_ = pathlib.Path(path)
    if path_.suffix == ".py":
        path_.parent.mkdir(parents=True, exist_ok=True)
    else:
        raise ValueError(f"Invalid file extension: {path_.suffix}, must be .py")
    if path_.is_file():
        raise FileExistsError(f"File {path} already exists")
    code = render_data_access_template(dataset.get_mock_path().name)
    with open(path, "w") as f:
        f.write(code)


In [19]:
init_code(dataset, "/Users/azizwork/Workspace/rds/notebooks/code1.py")

FileExistsError: File /Users/azizwork/Workspace/rds/notebooks/code1.py already exists

In [ ]:
ds_client.user_code